# RCSB PDBから取得するxmlから情報を取得する
https://www.rcsb.org/

In [1]:
import pandas as pd
from urllib.request import urlopen
import numpy as np
from lxml import etree
from Bio import SeqIO
from Bio.PDB import PDBList as pdbl
import urllib.parse
import urllib.request
import json

### Target名をUniprotで検索

In [58]:
target = input('target name?')
uniprot = 'https://www.uniprot.org/uniprot/?query={}&sort=score'.format(target)
list = pd.read_html(uniprot)[0]
list.head()

target name? Keap1


,Unnamed: 0,Entry,Entry name,Unnamed: 3,Protein names,Gene names,Organism,Length,e
0,NaN,Q14145,KEAP1_HUMAN,NaN,Kelch-like ECH-associated protein 1,"KEAP1 INRF2, KIAA0132, KLHL19",Homo sapiens (Human),624,NaN
1,NaN,Q9Z2X8,KEAP1_MOUSE,NaN,Kelch-like ECH-associated protein 1,"Keap1 Inrf2, Kiaa0132",Mus musculus (Mouse),624,NaN
2,NaN,P57790,KEAP1_RAT,NaN,Kelch-like ECH-associated protein 1,Keap1 Inrf2,Rattus norvegicus (Rat),624,NaN
3,NaN,Q684M4,KEAP1_PIG,NaN,Kelch-like ECH-associated protein 1,KEAP1,Sus scrofa (Pig),624,NaN
4,NaN,A0A2R8Q1W5,KEP1B_DANRE,NaN,Kelch-like ECH-associated protein 1...,keap1b,Danio rerio (Zebrafish) (Brachydanio rerio),587,NaN


### TargetのUniprotKB(Entry)を指定してxmlを取得

In [47]:
rowID = input('rowID? ')
UniprotKB = list['Entry'][int(rowID)]
handle = urlopen("http://www.uniprot.org/uniprot/{}.xml".format(UniprotKB))
record = SeqIO.read(handle, "uniprot-xml")
# print(record)

rowID?  0


### xmlの一部をDataFrameに変換

In [18]:
dbx = record.dbxrefs
dfIO = pd.DataFrame(dbx)
df = dfIO[0].str.split(':', expand=True)

### DataFrameにカラム名をつける

In [19]:
df.columns = ['name', 'id', 'and']
df

,name,id,and
0,ABCD,Q14145,None
1,Antibodypedia,1418,None
2,BMRB,Q14145,None
3,Bgee,ENSG00000079999,None
4,BindingDB,Q14145,None
...,...,...,...
318,VEuPathDB,HostDB,ENSG00000079999.13
319,eggNOG,KOG4441,None
320,iPTMnet,Q14145,None
321,jPOST,Q14145,None


### PDBのrowを指定してlistに格納

In [20]:
dfPDB = df[df['name']=='PDB']
PDBs = dfPDB['id'].values.tolist()
len(PDBs)

53

In [36]:
df

,name,id,and
0,ABCD,Q14145,None
1,Antibodypedia,1418,None
2,BMRB,Q14145,None
3,Bgee,ENSG00000079999,None
4,BindingDB,Q14145,None
...,...,...,...
318,VEuPathDB,HostDB,ENSG00000079999.13
319,eggNOG,KOG4441,None
320,iPTMnet,Q14145,None
321,jPOST,Q14145,None


### target名のdirectoryを作ってPDBを全てDL

In [10]:
for pdb_id in PDBs:
        pdbl().retrieve_pdb_file(pdb_id, pdir=target)
print('fertig')

fertig


### RCSB PDBでTargetを検索

In [14]:
rcsb_url = 'https://www.rcsb.org/structure/' + PDBs[0]
df = pd.read_html(rcsb_url)
df[0]

Entity ID: 1  \
                                            Molecule   
0                kelch-like ECH-associated protein 1   
1           UniProt & NIH Common Fund Data Resources   
2  Find proteins for&nbspQ14145&nbsp(Homo sapiens...   
3      PHAROS: &nbspQ14145GTEx: &nbspENSG00000079999   
4                         Protein Feature ViewExpand   
5  Reference SequencesetTimeout(function(){  Rcsb...   

                                                      \
                                              Chains   
0                                         A [auth X]   
1           UniProt & NIH Common Fund Data Resources   
2  Find proteins for&nbspQ14145&nbsp(Homo sapiens...   
3      PHAROS: &nbspQ14145GTEx: &nbspENSG00000079999   
4                         Protein Feature ViewExpand   
5  Reference SequencesetTimeout(function(){  Rcsb...   

                                                      \
                                     Sequence Length   
0                                                308   
1           UniProt & NIH Common Fund Data Resources   
2  Find proteins for&nbspQ14145&nbsp(Homo sapiens...   
3      PHAROS: &nbspQ14145GTEx: &nbspENSG00000079999   
4                         Protein Feature ViewExpand   
5  Reference SequencesetTimeout(function(){  Rcsb...   

                                                      \
                                            Organism   
0                                       Homo sapiens   
1           UniProt & NIH Common Fund Data Resources   
2  Find proteins for&nbspQ14145&nbsp(Homo sapiens...   
3      PHAROS: &nbspQ14145GTEx: &nbspENSG00000079999   
4                         Protein Feature ViewExpand   
5  Reference SequencesetTimeout(function(){  Rcsb...   

                                                      \
                                             Details   
0  Mutation(s): 0&nbspGene Names:&nbspKEAP1,&nbsp...   
1           UniProt & NIH Common Fund Data Resources   
2  Find proteins for&nbspQ14145&nbsp(Homo sapiens...   
3      PHAROS: &nbspQ14145GTEx: &nbspENSG00000079999   
4                         Protein Feature ViewExpand   
5  Reference SequencesetTimeout(function(){  Rcsb...   

                                                      
                                               Image  
0                                                NaN  
1           UniProt & NIH Common Fund Data Resources  
2  Find proteins for&nbspQ14145&nbsp(Homo sapiens...  
3      PHAROS: &nbspQ14145GTEx: &nbspENSG00000079999  
4                         Protein Feature ViewExpand  
5  Reference SequencesetTimeout(function(){  Rcsb...

### TargetのFASTAを取得

In [55]:
record.seq

Seq('MQPDPRPSGAGACCRFLPLQSQCPEGAGDAVMYASTECKAEVTPSQHGNRTFSY...CTC')

# 相同性を確認する

### ターゲットをUniprotで検索する

In [60]:
target = input('target name?')
uniprot = 'https://www.uniprot.org/uniprot/?query={}&sort=score'.format(target)
list = pd.read_html(uniprot)[0]
list.head()

target name? KCTD


,Unnamed: 0,Entry,Entry name,Unnamed: 3,Protein names,Gene names,Organism,Length,e
0,NaN,Q7L273,KCTD9_HUMAN,NaN,BTB/POZ domain-containing protein K...,KCTD9,Homo sapiens (Human),389,NaN
1,NaN,Q8N5Z5,KCD17_HUMAN,NaN,BTB/POZ domain-containing protein K...,KCTD17,Homo sapiens (Human),321,NaN
2,NaN,Q96MP8,KCTD7_HUMAN,NaN,BTB/POZ domain-containing protein K...,KCTD7,Homo sapiens (Human),289,NaN
3,NaN,Q8NC69,KCTD6_HUMAN,NaN,BTB/POZ domain-containing protein K...,KCTD6,Homo sapiens (Human),237,NaN
4,NaN,Q8WZ19,BACD1_HUMAN,NaN,BTB/POZ domain-containing adapter f...,"KCTD13 BACURD1, PDIP1, POLDIP1, FKSG86, PP6832",Homo sapiens (Human),329,NaN


### HumanにFilteringする

In [85]:
human = list[list['Organism'] == 'Homo sapiens (Human)']
human.head()

,Unnamed: 0,Entry,Entry name,Unnamed: 3,Protein names,Gene names,Organism,Length,e
0,NaN,Q7L273,KCTD9_HUMAN,NaN,BTB/POZ domain-containing protein K...,KCTD9,Homo sapiens (Human),389,NaN
1,NaN,Q8N5Z5,KCD17_HUMAN,NaN,BTB/POZ domain-containing protein K...,KCTD17,Homo sapiens (Human),321,NaN
2,NaN,Q96MP8,KCTD7_HUMAN,NaN,BTB/POZ domain-containing protein K...,KCTD7,Homo sapiens (Human),289,NaN
3,NaN,Q8NC69,KCTD6_HUMAN,NaN,BTB/POZ domain-containing protein K...,KCTD6,Homo sapiens (Human),237,NaN
4,NaN,Q8WZ19,BACD1_HUMAN,NaN,BTB/POZ domain-containing adapter f...,"KCTD13 BACURD1, PDIP1, POLDIP1, FKSG86, PP6832",Homo sapiens (Human),329,NaN


In [63]:
rowID = input('rowID? ')
KCTD17 = list['Entry'][int(rowID)]
handle = urlopen("http://www.uniprot.org/uniprot/{}.xml".format(KCTD17))
record = SeqIO.read(handle, "uniprot-xml")
print(record)

rowID?  1


ID: Q8N5Z5
Name: KCD17_HUMAN
Description: BTB/POZ domain-containing protein KCTD17
Database cross-references: Antibodypedia:11878, Bgee:ENSG00000100379, BioGRID-ORCS:79734, BioGRID:122848, BioMuta:KCTD17, CCDS:CCDS13940.2, CCDS:CCDS74854.1, CTD:79734, DMDM:205371782, DNASU:79734, DOI:10.1016/j.ajhg.2015.04.008, DOI:10.1038/990031, DOI:10.1038/ncomms6081, DOI:10.1038/ng1285, DOI:10.1042/bcj20170527, DOI:10.1101/gr.2596504, DisGeNET:79734, EMBL:AK022304, EMBL:AL022314, EMBL:BC025403, EMBL:BC031038, EMBL:CH471095, EPD:Q8N5Z5, Ensembl:ENST00000402077, Ensembl:ENST00000403888, ExpressionAtlas:Q8N5Z5, GO:GO:0005737, GO:GO:0005783, GO:GO:0030030, GO:GO:0031463, GO:GO:0032469, GO:GO:0042802, GO:GO:0043161, GO:GO:0045724, GO:GO:0051260, GO:GO:0097602, Gene3D:3.30.710.10, GeneCards:KCTD17, GeneID:79734, GeneTree:ENSGT00940000161746, Genevisible:Q8N5Z5, GenomeRNAi:79734, HGNC:HGNC:25705, HOGENOM:CLU_070830_2_0_1, HPA:ENSG00000100379, InParanoid:Q8N5Z5, IntAct:Q8N5Z5, InterPro:IPR000210, InterPro: